In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0
!pip install mrcnn

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!unrar x "/content/gdrive/MyDrive/Treino/archive.rar"

In [ ]:
import warnings
from os import listdir
from numpy import zeros
from numpy import asarray
from matplotlib import pyplot
from numpy import expand_dims
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from xml.etree import ElementTree
from keras.engine import topology
from mrcnn.model import mold_image
from matplotlib.patches import Rectangle

warnings.filterwarnings('ignore')
 
class CarPlateDataset(Dataset):
  # Carrega o dataset
  def load_dataset(self, dataset_dir, is_train=True):
    self.add_class("dataset", 1, "carPlate")

    images_dir = dataset_dir + '/images/'
    annotations_dir = dataset_dir + '/annotations/'

    for filename in listdir(images_dir):
      small_name = filename.replace("Cars", "").replace(".png", "")

      if small_name in ['00090']:
        continue
      if is_train and int(small_name) >= 30:
        continue
      if not is_train and int(small_name) < 30:
        continue

      img_path = images_dir + filename
      ann_path = annotations_dir + filename.replace(".png", "") + '.xml'
      self.add_image('dataset', image_id=small_name, path=img_path, annotation=ann_path)
 
  # Obtém a posição da placa através do xml
  def extract_boxes(self, filename):
    tree = ElementTree.parse(filename)
    root = tree.getroot()
    boxes = list()

    for box in root.findall('.//bndbox'):
      xmin = int(box.find('xmin').text)
      ymin = int(box.find('ymin').text)
      xmax = int(box.find('xmax').text)
      ymax = int(box.find('ymax').text)
      coors = [xmin, ymin, xmax, ymax]
      boxes.append(coors)

    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height
 
  # Carrega um overlay branco sobre a posição da placa na imagem
  def load_mask(self, image_id):
    info = self.image_info[image_id]
    path = info['annotation']
    boxes, w, h = self.extract_boxes(path)
    masks = zeros([h, w, len(boxes)], dtype='uint8')
    class_ids = list()

    for i in range(len(boxes)):
      box = boxes[i]
      row_s, row_e = box[1], box[3]
      col_s, col_e = box[0], box[2]
      masks[row_s:row_e, col_s:col_e, i] = 1
      class_ids.append(self.class_names.index('carPlate'))

    return masks, asarray(class_ids, dtype='int32')
 
  def image_reference(self, image_id):
    info = self.image_info[image_id]
    return info['path']

# Plota a imagem com a mascara na posição correta e com a predição realizada
def plot_actual_vs_predicted(dataset, model, cfg, n_images=5):
  pyplot.figure(figsize=(100, 100), dpi=80)

  for i in range(n_images):
    image = dataset.load_image(i)
    mask, _ = dataset.load_mask(i)
    scaled_image = mold_image(image, cfg)
    sample = expand_dims(scaled_image, 0)
    yhat = model.detect(sample, verbose=0)[0]
    pyplot.subplot(n_images, 2, i*2+1)
    pyplot.imshow(image)
    pyplot.title('Actual')

    for j in range(mask.shape[2]):
      pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)

    pyplot.subplot(n_images, 2, i*2+2)
    pyplot.imshow(image)
    pyplot.title('Predicted')
    ax = pyplot.gca()

    for box in yhat['rois']:
      y1, x1, y2, x2 = box
      width, height = x2 - x1, y2 - y1
      rect = Rectangle((x1, y1), width, height, fill=False, color='red')
      ax.add_patch(rect)

  pyplot.show()

class TrainConfig(Config):
	NAME = "car_plate_cfg"
	NUM_CLASSES = 1 + 1
	STEPS_PER_EPOCH = 40

class PredictionConfig(Config):
  NAME = "car_plate_cfg"
  NUM_CLASSES = 1 + 1
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1


train_set = CarPlateDataset()
train_set.load_dataset('./', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
 
test_set = CarPlateDataset()
test_set.load_dataset('./', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

# Treinamento
config = TrainConfig()
model = MaskRCNN(mode='training', model_dir='/content/gdrive/MyDrive/Treino/models', config=config)
model.load_weights('/content/gdrive/MyDrive/Treino/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=2, layers='heads')

# Teste
cfg = PredictionConfig()
predict_model = MaskRCNN(mode='inference', model_dir='/content/gdrive/MyDrive/Treino/models', config=cfg)
predict_model.load_weights('/content/gdrive/MyDrive/Treino/models/car_plate_cfg20211204T1512/mask_rcnn_car_plate_cfg_0001.h5', by_name=True)
plot_actual_vs_predicted(train_set, predict_model, cfg)
plot_actual_vs_predicted(test_set, predict_model, cfg)